In [1]:
from sodapy import Socrata
import geopandas as gpd
import json
from keplergl import KeplerGl

In [2]:
#insert your own token to avoid throttling
#found at https://data.cityofchicago.org/profile
token = 'OfytEmI12Fl0Yq13HTVq7ROlF'

In [3]:
# creating the daterange we'll use for this notebook
start = '2019-03-01'
end = '2019-03-31'

In [4]:
def get_rides(start, end, token=None):
    '''
    Creates a request for the SODA API datasets. 
    
    returns: geopandas coordinates for rides
    '''
    #instantiate the Socrata client
    client = Socrata(
        domain = 'data.cityofchicago.org',
        app_token=token, timeout=100000
    )
    
    #set up the SoSQL call
    select = 'pickup_centroid_location AS geometry, COUNT(trip_id) AS rides'
    group = 'geometry' 
    where = f'trip_start_timestamp between "{start}" and "{end}"'
    
    #make the call for the data, specifying the correct dataset
    results = client.get(
        dataset_identifier='m6dm-c72p',
        select=select, group=group, where=where,
        content_type = 'geojson', limit=100_000
    )
    client.close()
    
    #add data to geopandas
    gdf = gpd.GeoDataFrame.from_features(results).dropna()
    gdf.rides = gdf.rides.astype('int32')
    
    return gdf

In [5]:
# creating the daterange we'll use for this notebook
start = '2019-03-01'
end = '2019-03-31'

In [6]:
%%time
#calling our data gathering funciton
gdf_point = get_rides(start, end, token)

CPU times: user 67.2 ms, sys: 12.1 ms, total: 79.3 ms
Wall time: 37.1 s


In [7]:
#defining helper functions to load/save map config files
#(to maintain similar visuals)

def open_config(file_name):
    '''
    helper function to load preconfigured Kepler configs
    '''
    try: 
        current_file = json.load(open(file_name, 'r'))
        print('config loaded')
        return current_file
    except FileNotFoundError:
        print('no config file found. plz initialize map, and use map.config')
        return {}

    
def save_config(config, file_name):
    '''
    helper function to save the current config for Kepler
    '''
    def save():
        with open(file_name, 'w') as f:
                json.dump(config, f)
    try:
        current_file = json.load(open(file_name, 'r'))
        if config != current_file:
            print('changes detected. saving current config')
            save()
        else:
            print('no changes detected')
            
    except FileNotFoundError:
        print('no existing file in directory. saving current config')
        save()

In [8]:
point_config = open_config('configs/point_config.json')

config loaded


In [44]:
map_1 = KeplerGl(height=900, config=point_config)
map_1.add_data(gdf_point, 'rides')
map_1

User Guide: https://github.com/keplergl/kepler.gl/blob/master/docs/keplergl-jupyter/user-guide.md


KeplerGl(config={'version': 'v1', 'config': {'visState': {'filters': [], 'layers': [{'id': 'y3o56bm', 'type': …

In [45]:
save_config(map_1.config, file_name='configs/point_config.json')

no changes detected


In [46]:
del map_1

In [12]:
def get_boundaries(token=None):
    '''
    Creates a request for the SODA API datasets. 
    area inputs:
        'community' - community areas
        'census' - census tracts
        
    returns: geopandas coordinates for boundaries
    '''
    #instantiate the Socrata client with the domain
    #optional to input your own app_token to bypass throttling
    client = Socrata(
        domain = 'data.cityofchicago.org', app_token=token
    )
        
    #only return what is needed and simplify the geometry
    #(to minimize download size)
    select = 'geoid10 as area,\
    simplify_preserve_topology(the_geom, 0.0001) AS the_geom'
    
    #make the request
    results = client.get(
        dataset_identifier = '74p9-q2aq', content_type = 'geojson',
        select = select
    )
    
    #turn results into a GeoDataFrame
    gdf = gpd.GeoDataFrame.from_features(results)
    
    return gdf.dropna()

In [13]:
bounds = get_boundaries(token)

In [14]:
bounds_config = open_config('configs/bounds_config.json')

config loaded


In [47]:
map_2 = KeplerGl(height=800, config=bounds_config)
map_2.add_data(bounds, 'boundaries')
map_2

User Guide: https://github.com/keplergl/kepler.gl/blob/master/docs/keplergl-jupyter/user-guide.md


KeplerGl(config={'version': 'v1', 'config': {'visState': {'filters': [], 'layers': [{'id': 'viq97cc', 'type': …

In [16]:
save_config(map_2.config, 'configs/bounds_config.json')

no changes detected


In [48]:
del map_2

In [18]:
gb_config = open_config('configs/gb_config.json')

config loaded


In [49]:
map_3 = KeplerGl(height=800, config=gb_config)
map_3.add_data(bounds, 'boundaries')
map_3.add_data(gdf_point, 'rides')
map_3

User Guide: https://github.com/keplergl/kepler.gl/blob/master/docs/keplergl-jupyter/user-guide.md


KeplerGl(config={'version': 'v1', 'config': {'visState': {'filters': [], 'layers': [{'id': 'pakdrbr', 'type': …

In [20]:
save_config(map_3.config, 'configs/gb_config.json')

no changes detected


In [50]:
del map_3

In [22]:
bounds.geometry[0].contains(gdf_point.geometry[0])

False

In [23]:
def get_num_rides(bound, points):
    
    tup = points.itertuples()
    
    lst = [ride.rides for ride in tup if bound.contains(ride.geometry)]
    
    return sum(lst)

In [24]:
# gdf_point.rides = gdf_point.rides.astype('int32')

In [25]:
bounds['rides'] = bounds.geometry.map(lambda area: get_num_rides(area, gdf_point))

In [26]:
br_config = open_config('configs/br_config.json')

config loaded


In [51]:
map_4 = KeplerGl(height=700, config=br_config)
map_4.add_data(bounds, 'bounds')
map_4

User Guide: https://github.com/keplergl/kepler.gl/blob/master/docs/keplergl-jupyter/user-guide.md


KeplerGl(config={'version': 'v1', 'config': {'visState': {'filters': [], 'layers': [{'id': 't0oktdn', 'type': …

In [28]:
save_config(map_4.config, 'configs/br_config.json')

no changes detected


In [52]:
del map_4

In [30]:
def get_daily_rides(start, end, token=None):
    '''
    Creates a request for the SODA API datasets. 
    
    returns: geopandas coordinates for rides
    '''
    #instantiate the Socrata client
    client = Socrata(
        domain = 'data.cityofchicago.org',
        app_token=token, timeout=100000
    )
    
    #set up the SoSQL call
    select = 'date_trunc_ymd(trip_start_timestamp) as date,\
    pickup_centroid_location AS geometry, COUNT(trip_id) AS rides'
    group = 'date, geometry'
    where = f'date between "{start}" and "{end}"'
    
    #make the call for the data, specifying the correct dataset
    results = client.get(
        dataset_identifier='m6dm-c72p',
        select=select, group=group, where=where,
        content_type = 'geojson', limit=100_000
    )
    client.close()
    
    #add data to geopandas
    gdf = gpd.GeoDataFrame.from_features(results).dropna()
    gdf.rides = gdf.rides.astype('int32')
    
    return gdf

In [31]:
%%time
gdf_daily = get_daily_rides(start, end, token)

CPU times: user 519 ms, sys: 32 ms, total: 551 ms
Wall time: 4min 57s


In [32]:
def get_num_rides_daily(points, bounds):
    
    daily_bounds = []

    for date in points.date.unique():
        df = points[points['date'] == date].copy()

        for geo in bounds.geometry:
            tup = df.itertuples()
            rides = sum(
                [point.rides for point in tup if geo.contains(point.geometry)]
            )
            daily_bounds.append([date, geo, rides])
            
    daily_bounds = gpd.GeoDataFrame(
        daily_bounds, columns=['date','geometry','rides']
    )
    
    return daily_bounds

In [33]:
daily_bounds = get_num_rides_daily(gdf_daily, bounds)

In [39]:
db_config = open_config('configs/db_config.json')

config loaded


In [54]:
map_5 = KeplerGl(height=800, config=db_config)
map_5.add_data(daily_bounds, 'daily_trips')
map_5

User Guide: https://github.com/keplergl/kepler.gl/blob/master/docs/keplergl-jupyter/user-guide.md


KeplerGl(config={'version': 'v1', 'config': {'visState': {'filters': [{'dataId': 'daily_trips', 'id': 'ltpbcfv…

In [55]:
save_config(map_5.config, 'configs/db_config.json')

changes detected. saving current config


In [56]:
map_5.save_to_html(data=map_5.data, file_name='chicago.html')

Map saved to chicago.html!


In [57]:
del map_5